In [1]:
# ===========================================
# 1️⃣ Importar librerías
# ===========================================
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

# ===========================================
# 2️⃣ Cargar el dataset MNIST
# ===========================================
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalizar (convertir a valores entre 0 y 1)
x_train, x_test = x_train / 255.0, x_test / 255.0

# Agregar dimensión de canal (necesario para Conv2D)
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# Crear datasets de entrenamiento y prueba
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# ===========================================
# 3️⃣ Definir el modelo CNN
# ===========================================
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Crear instancia del modelo
model = MyModel()

# ===========================================
# 4️⃣ Definir pérdida, optimizador y métricas
# ===========================================
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# ===========================================
# 5️⃣ Definir las funciones de entrenamiento y prueba
# ===========================================
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss.update_state(loss)
    train_accuracy.update_state(labels, predictions)

@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    test_loss.update_state(t_loss)
    test_accuracy.update_state(labels, predictions)

# ===========================================
# 6️⃣ Entrenar el modelo
# ===========================================
EPOCHS = 5

for epoch in range(EPOCHS):
    # Entrenamiento
    for images, labels in train_ds:
        train_step(images, labels)

    # Evaluación
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    # Mostrar resultados
    template = 'Época {}, Pérdida: {:.4f}, Exactitud: {:.2f}%, Pérdida de prueba: {:.4f}, Exactitud de prueba: {:.2f}%'
    print(template.format(
        epoch + 1,
        train_loss.result(),
        train_accuracy.result() * 100,
        test_loss.result(),
        test_accuracy.result() * 100
    ))

    # Reiniciar métricas
    train_loss.reset_state()
    train_accuracy.reset_state()
    test_loss.reset_state()
    test_accuracy.reset_state()


print("\n✅ Entrenamiento completado.")


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Época 1, Pérdida: 0.1331, Exactitud: 95.97%, Pérdida de prueba: 0.0658, Exactitud de prueba: 97.73%
Época 2, Pérdida: 0.0419, Exactitud: 98.69%, Pérdida de prueba: 0.0532, Exactitud de prueba: 98.25%
Época 3, Pérdida: 0.0220, Exactitud: 99.32%, Pérdida de prueba: 0.0599, Exactitud de prueba: 98.18%
Época 4, Pérdida: 0.0136, Exactitud: 99.53%, Pérdida de prueba: 0.0619, Exactitud de prueba: 98.31%
Época 5, Pérdida: 0.0095, Exactitud: 99.69%, Pérdida de prueba: 0.0663, Exactitud de prueba: 98.21%

✅ Entrenamiento completado.
